# Quantum-inspired baseline (PennyLane + PyTorch)

Lightweight demo to benchmark a small hybrid quantum-classical model on Obfuscated-MalMem2022.

**Notes**
- Designed for ~16GB RAM: samples a small balanced subset and reduces features with PCA.
- Runs fully on CPU using PennyLane default.qubit; no real QPU needed.
- Goal: get a feel for viability vs. classical baselines (accuracy/F1), not to beat SOTA.

In [1]:
# If needed, install deps (uncomment):
!pip install pennylane torch scikit-learn pandas numpy

  Using cached pennylane-0.43.1-py3-none-any.whl.metadata (11 kB)
  Using cached torch-2.9.1-cp312-cp312-manylinux_2_28_x86_64.whl.metadata (30 kB)
  Using cached networkx-3.6-py3-none-any.whl.metadata (6.8 kB)
  Using cached rustworkx-0.17.1-cp39-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (10 kB)
  Using cached autograd-1.8.0-py3-none-any.whl.metadata (7.5 kB)
  Using cached appdirs-1.4.4-py2.py3-none-any.whl.metadata (9.0 kB)
  Using cached autoray-0.8.0-py3-none-any.whl.metadata (6.1 kB)
  Using cached pennylane_lightning-0.43.0-cp312-cp312-manylinux_2_27_x86_64.manylinux_2_28_x86_64.whl.metadata (11 kB)
  Using cached tomlkit-0.13.3-py3-none-any.whl.metadata (2.8 kB)
  Using cached diastatic_malt-2.15.2-py3-none-any.whl.metadata (2.6 kB)
  Using cached setuptools-80.9.0-py3-none-any.whl.metadata (6.6 kB)
  Using cached sympy-1.14.0-py3-none-any.whl.metadata (12 kB)
  Using cached fsspec-2025.10.0-py3-none-any.whl.metadata (10 kB)
  Using cached nvidia_cuda_nvrtc_c

In [1]:
import numpy as np
import pandas as pd
from pathlib import Path
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.metrics import classification_report

import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader
import pennylane as qml

In [15]:
# Config
DATA_PATH = Path("Obfuscated-MalMem2022.csv")
if not DATA_PATH.exists():
    DATA_PATH = Path("..").joinpath("Obfuscated-MalMem2022.csv")
N_SAMPLES_PER_CLASS = 10000  # small to fit in 16GB; adjust as needed
BATCH_SIZE = 32
EPOCHS = 30
LR = 1e-3
N_QUBITS = 4  # keep small; matches PCA components

# Load
df = pd.read_csv(DATA_PATH)
df['label'] = df['Class'].apply(lambda c: 0 if str(c).lower() == 'benign' else 1)

# Balanced small subset
subset = (
    df.groupby('label', group_keys=False)
    .apply(lambda g: g.sample(min(len(g), N_SAMPLES_PER_CLASS), random_state=42))
    .sample(frac=1.0, random_state=42)
)

X = subset.drop(columns=['Class', 'Category', 'label'], errors='ignore').to_numpy(dtype=np.float32)
y = subset['label'].to_numpy(dtype=np.int64)

# Scale + reduce
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
pca = PCA(n_components=N_QUBITS, random_state=42)
X_enc = pca.fit_transform(X_scaled).astype(np.float32)

X_train, X_test, y_train, y_test = train_test_split(
    X_enc, y, test_size=0.2, random_state=42, stratify=y
)

train_ds = TensorDataset(torch.tensor(X_train), torch.tensor(y_train))
test_ds = TensorDataset(torch.tensor(X_test), torch.tensor(y_test))
train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(test_ds, batch_size=BATCH_SIZE)

print("Subset size", len(subset), "Train", len(train_ds), "Test", len(test_ds))

/tmp/ipykernel_38458/1134695053.py:18: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda g: g.sample(min(len(g), N_SAMPLES_PER_CLASS), random_state=42))


Subset size 20000 Train 16000 Test 4000


In [3]:
# Quantum layer via PennyLane TorchLayer
dev = qml.device("default.qubit", wires=N_QUBITS)

def angle_encoding(x, wires):
    for i, w in enumerate(wires):
        qml.RX(x[i], wires=w)

def variational_block(weights, wires):
    # Simple hardware-efficient ansatz
    for i, w in enumerate(wires):
        qml.RY(weights[i], wires=w)
    for i in range(len(wires) - 1):
        qml.CNOT(wires=[wires[i], wires[i + 1]])
    qml.CNOT(wires=[wires[-1], wires[0]])

def qnode_fn(inputs, weights):
    angle_encoding(inputs, wires=range(N_QUBITS))
    variational_block(weights, wires=range(N_QUBITS))
    return [qml.expval(qml.PauliZ(i)) for i in range(N_QUBITS)]

weight_shapes = {"weights": (N_QUBITS,)}
qnode = qml.QNode(qnode_fn, dev, interface="torch", diff_method="backprop")
qlayer = qml.qnn.TorchLayer(qnode, weight_shapes)

In [4]:
class HybridModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.classical = nn.Sequential(
            nn.Linear(N_QUBITS, N_QUBITS),
            nn.ReLU(),
        )
        self.quantum = qlayer
        self.head = nn.Linear(N_QUBITS, 1)

    def forward(self, x):
        x = self.classical(x)
        # TorchLayer in this version doesn't batch internally; loop over batch
        q_out = torch.stack([self.quantum(xi) for xi in x])
        x = self.head(q_out)
        return torch.sigmoid(x)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = HybridModel().to(device)
opt = torch.optim.Adam(model.parameters(), lr=LR)
loss_fn = nn.BCELoss()

print(model)

HybridModel(
  (classical): Sequential(
    (0): Linear(in_features=4, out_features=4, bias=True)
    (1): ReLU()
  )
  (quantum): <Quantum Torch Layer: func=qnode_fn>
  (head): Linear(in_features=4, out_features=1, bias=True)
)


In [5]:
def train_epoch(loader):
    model.train()
    total_loss = 0.0
    for xb, yb in loader:
        xb, yb = xb.to(device), yb.to(device).float().unsqueeze(1)
        opt.zero_grad()
        preds = model(xb)
        loss = loss_fn(preds, yb)
        loss.backward()
        opt.step()
        total_loss += loss.item() * len(xb)
    return total_loss / len(loader.dataset)

def eval_epoch(loader):
    model.eval()
    all_preds, all_labels = [], []
    with torch.no_grad():
        for xb, yb in loader:
            xb = xb.to(device)
            preds = model(xb).cpu().numpy().ravel()
            all_preds.extend(preds)
            all_labels.extend(yb.numpy())
    all_preds = np.array(all_preds)
    all_labels = np.array(all_labels)
    pred_labels = (all_preds >= 0.5).astype(int)
    report = classification_report(all_labels, pred_labels, target_names=["benign", "malware"], output_dict=True, zero_division=0)
    return report

for epoch in range(1, EPOCHS + 1):
    train_loss = train_epoch(train_loader)
    report = eval_epoch(test_loader)
    print(f"Epoch {epoch}: loss={train_loss:.4f} test_acc={report['accuracy']:.4f} f1={report['weighted avg']['f1-score']:.4f}")


Epoch 1: loss=0.7180 test_acc=0.5000 f1=0.3333
Epoch 2: loss=0.7078 test_acc=0.5000 f1=0.3333
Epoch 3: loss=0.7020 test_acc=0.5000 f1=0.3333
Epoch 4: loss=0.6986 test_acc=0.5000 f1=0.3333
Epoch 5: loss=0.6957 test_acc=0.5000 f1=0.3333
Epoch 6: loss=0.6920 test_acc=0.5000 f1=0.3333
Epoch 7: loss=0.6844 test_acc=0.4775 f1=0.3273
Epoch 8: loss=0.6659 test_acc=0.6825 f1=0.6609
Epoch 9: loss=0.6286 test_acc=0.8525 f1=0.8518
Epoch 10: loss=0.5725 test_acc=0.8975 f1=0.8974
Epoch 11: loss=0.5105 test_acc=0.9050 f1=0.9050
Epoch 12: loss=0.4548 test_acc=0.9075 f1=0.9075
Epoch 13: loss=0.4104 test_acc=0.9200 f1=0.9200
Epoch 14: loss=0.3762 test_acc=0.9275 f1=0.9275
Epoch 15: loss=0.3500 test_acc=0.9275 f1=0.9275
Epoch 16: loss=0.3299 test_acc=0.9275 f1=0.9275
Epoch 17: loss=0.3114 test_acc=0.9250 f1=0.9250
Epoch 18: loss=0.2964 test_acc=0.9325 f1=0.9325
Epoch 19: loss=0.2845 test_acc=0.9325 f1=0.9325
Epoch 20: loss=0.2752 test_acc=0.9325 f1=0.9325
Epoch 21: loss=0.2670 test_acc=0.9325 f1=0.9325
E

In [6]:
# Final report
final_report = eval_epoch(test_loader)
pd.DataFrame(final_report).T

,precision,recall,f1-score,support
benign,0.953368,0.9200,0.936387,200.0000
malware,0.922705,0.9550,0.938575,200.0000
accuracy,0.937500,0.9375,0.937500,0.9375
macro avg,0.938037,0.9375,0.937481,400.0000
weighted avg,0.938037,0.9375,0.937481,400.0000


## Quantum variants: QMLP and QCNN
Lightweight circuits for comparison. Keep epochs small to fit 16GB RAM.

In [12]:
# Builders for quantum MLP and quantum CNN-style circuits

import pennylane as qml

import torch.nn as nn


def make_qmlp_layer(n_qubits, layers=1):
    dev = qml.device('default.qubit', wires=n_qubits)

    @qml.qnode(dev, interface='torch', diff_method='backprop')
    def circuit(inputs, weights):
        qml.AngleEmbedding(inputs, wires=range(n_qubits))
        qml.BasicEntanglerLayers(weights, wires=range(n_qubits))
        return [qml.expval(qml.PauliZ(i)) for i in range(n_qubits)]

    weight_shapes = {'weights': (layers, n_qubits)}
    return qml.qnn.TorchLayer(circuit, weight_shapes)


def make_qcnn_layer(n_qubits, layers=1):
    dev = qml.device('default.qubit', wires=n_qubits)

    @qml.qnode(dev, interface='torch', diff_method='backprop')
    def circuit(inputs, weights):
        qml.AngleEmbedding(inputs, wires=range(n_qubits))
        # simple conv-like entangling: pairwise CX + RY
        for l in range(layers):
            for i in range(n_qubits):
                qml.CNOT(wires=[i, (i+1)%n_qubits])
                qml.RY(weights[l, i], wires=i)
        return [qml.expval(qml.PauliZ(i)) for i in range(n_qubits)]

    weight_shapes = {'weights': (layers, n_qubits)}
    return qml.qnn.TorchLayer(circuit, weight_shapes)


def train_and_eval(model, epochs=4, lr=1e-3):
    opt = torch.optim.Adam(model.parameters(), lr=lr)
    loss_fn = nn.BCELoss()

    def _loop(loader, train=True):
        if train:
            model.train()
        else:
            model.eval()
        total_loss = 0.0
        all_preds, all_labels = [], []
        with torch.set_grad_enabled(train):
            for xb, yb in loader:
                xb = xb.to(device)
                yb = yb.to(device).float().unsqueeze(1)
                if train:
                    opt.zero_grad()
                preds = model(xb)
                loss = loss_fn(preds, yb)
                if train:
                    loss.backward()
                    opt.step()
                total_loss += loss.item() * len(xb)
                all_preds.extend(preds.detach().cpu().numpy().ravel())
                all_labels.extend(yb.cpu().numpy().ravel())
        return total_loss / max(len(loader.dataset), 1), np.array(all_preds), np.array(all_labels)

    for _ in range(epochs):
        train_loss, _, _ = _loop(train_loader, train=True)

    _, preds, labels = _loop(test_loader, train=False)
    pred_labels = (preds >= 0.5).astype(int)
    report = classification_report(
        labels, pred_labels, target_names=['benign', 'malware'], output_dict=True, zero_division=0
    )
    return report


In [16]:
# Compare QMLP and QCNN
def make_qmlp_model():
    qlayer = make_qmlp_layer(N_QUBITS, layers=1)
    class Model(nn.Module):
        def __init__(self):
            super().__init__(); self.q = qlayer; self.head = nn.Linear(N_QUBITS,1)
        def forward(self, x):
            q_out = torch.stack([self.q(xi) for xi in x])
            return torch.sigmoid(self.head(q_out))
    return Model().to(device)

def make_qcnn_model():
    qlayer = make_qcnn_layer(N_QUBITS, layers=1)
    class Model(nn.Module):
        def __init__(self):
            super().__init__(); self.q = qlayer; self.head = nn.Linear(N_QUBITS,1)
        def forward(self, x):
            q_out = torch.stack([self.q(xi) for xi in x])
            return torch.sigmoid(self.head(q_out))
    return Model().to(device)

for name, builder in [('qmlp', make_qmlp_model), ('qcnn', make_qcnn_model)]:
    report = train_and_eval(builder(), epochs=5, lr=1e-3)
    print(name, 'accuracy', report['accuracy'], 'f1', report['weighted avg']['f1-score'])
    display(pd.DataFrame(report).T)


qmlp accuracy 0.755 f1 0.7547937815703005


,precision,recall,f1-score,support
benign,0.770701,0.726,0.747683,2000.000
malware,0.741021,0.784,0.761905,2000.000
accuracy,0.755000,0.755,0.755000,0.755
macro avg,0.755861,0.755,0.754794,4000.000
weighted avg,0.755861,0.755,0.754794,4000.000


qcnn accuracy 0.5985 f1 0.5980622898336289


,precision,recall,f1-score,support
benign,0.605460,0.5655,0.584798,2000.0000
malware,0.592402,0.6315,0.611326,2000.0000
accuracy,0.598500,0.5985,0.598500,0.5985
macro avg,0.598931,0.5985,0.598062,4000.0000
weighted avg,0.598931,0.5985,0.598062,4000.0000
